# ONS Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
from datetime import date, datetime

import common_core

## Configuration

Data to download from the NHS statistical work area

In [2]:
# The 4 nations in the UK
nationNames = [common_core.ENGLAND, common_core.SCOTLAND, common_core.WALES, common_core.NORTHERN_IRELAND]

# The 9 regions in England
regionNames = common_core.regionNames

# The aliases for regions in England
regionAliases = {"East of England": ["East"]}

# Combine all of these area types into a single list
areas = [("nation", nationNames), ("region", regionNames)]

In [3]:
# The latest ONS age bands
ageDemographics = [
    '<1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29',
    '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+'
]

# The legacy ONS age bands
legacyAgeDemographics = [
    '01-14', '15-44', '45-64', '65-74', '75-84', '85+'
]

## Constants

Text strings to avoid hard-coded values throughout the code; avoids clutter and silent errors.

In [4]:
# Index from Fri 2 Jan 1970
minWeek = date(1970, 1, 2)
maxWeek = date(datetime.now().year, 12, 31)

In [5]:
# Weeks have an end date and a number, always ending on Fridays
WEEK_ENDED = "week_ended"
WEEK_NUMBER = "week_number"

# Internal names used by the cache
WEEK_COL_NOS = "week_col_nos"
WEEK_NUMBERS = "week_numbers"
WEEK_ENDINGS = "week_endings"
WEEK_OFFSETS = "week_offsets"

# Deaths are reported by registration date and occurrence date
TOTAL_REGISTRATIONS = "total_registrations"
TOTAL_OCCURRENCES = "total_occurrences"
COVID_REGISTRATIONS = "covid_registrations"
COVID_OCCURRENCES = "covid_occurrences"

## Interactive Testing

In [6]:
if __name__ == '__main__':
    pass